In [2]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np
import scipy.optimize as so
import math

# ==========================================================================
#  Parametri dello script
N_acquisizioni = 10   # numero di acquisizioni da eseguire
nper = 2             # numero periodi usati per la stima
npt = 8192           # numero MASSIMO di punti acquisiti
nf = 200             # numero di frequenze nello sweep da f0 a f1
f0 = 10
f1 = 1e3
flag_return = False  # A/R? o solo andata?
flag_show = True     # Visualizza i segnali nel tempo?
# vettore delle frequenze
fv = np.logspace(np.log10(f0), np.log10(f1), nf)

# ==========================================================================
#  Funzioni di utilità
def circolare_media_var_della_media(theta_mat):
    """
    theta_mat: array shape (N_acq, nf) con angoli in radianti, ammessi NaN.
    Ritorna: mean (nf,), var_mean (nf,)
      - mean: media circolare in [-pi, pi]
      - var_mean: varianza campionaria della media (angolare) [rad^2]
    """
    # conteggi validi per frequenza
    valid_counts = np.sum(~np.isnan(theta_mat), axis=0).astype(float)

    # media vettoriale <cos>, <sin> ignorando NaN
    C = np.nanmean(np.cos(theta_mat), axis=0)
    S = np.nanmean(np.sin(theta_mat), axis=0)

    # media circolare
    mean_ang = np.arctan2(S, C)

    # residui avvolti rispetto alla media in [-pi, pi]
    resid = np.angle(np.exp(1j * (theta_mat - mean_ang)))

    # varianza campionaria dei residui (serve almeno 2 validi)
    var_resid = np.nanvar(resid, axis=0, ddof=1)

    # varianza della media = var_resid / N_eff
    with np.errstate(invalid='ignore', divide='ignore'):
        var_mean = var_resid / valid_counts

    # se <2 validi -> NaN
    var_mean[valid_counts < 2] = np.nan

    return mean_ang, var_mean

# ==========================================================================
#  Configurazione base AD2 (più parametri impostati dopo)
ad2 = tdwf.AD2()
ad2.vdd = 3
ad2.vss = -3
ad2.power(True)

wavegen = tdwf.WaveGen(ad2.hdwf)
wavegen.w1.ampl = 0.01
wavegen.w1.func = tdwf.funcSine
wavegen.w1.start()

scope = tdwf.Scope(ad2.hdwf)
scope.ch1.rng = 5
scope.ch2.rng = 5

# ==========================================================================
#   Esecuzione...

# [1] Crea plot
fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(2, 2, figsize=(12, 6),
    gridspec_kw={'width_ratios': [1, 2]})
fig.canvas.manager.set_window_title('Spazzata frequenza')

# [2] Preparazione dataset
flag_first = True  # primo plot diverso dai seguenti (update)

# Liste per accogliere i dati ad ogni acquisizione
matrice_risultati_ampiezza = []     # shape finale: (N_acquisizioni, nf)
matrice_risultati_fase = []         # shape finale: (N_acquisizioni, nf)

# CICLO FOR PER ESEGUIRE N_ACQUISIZIONI
for k in range(N_acquisizioni):

    # Messaggio per tenere traccia dell'avanzamento
    print(f"--- Acquisizione {k+1} di {N_acquisizioni} ---")

    # Resetto i vettori per ogni acquisizione
    Am = np.full((nf, 2), np.nan)    # pre-allocazione dati ampiezza
    phim = np.full((nf, 2), np.nan)  # pre-allocazione dati fase

    # [3] Ciclo misura
    for ar in range(2 if flag_return else 1):  # Ciclo A/R
        for ii in range(len(fv)):  # Ciclo frequenze
            # [3a] calcolo indice frequenza
            if ar == 0:  # caso A(ndata)
                findex = ii
            else:        # caso R(itorno)
                findex = len(fv) - ii - 1

            # Frequenza attuale
            ff = fv[findex]

            # [3b] stima parametri di sampling
            #   Voglio misurare nper periodi a ff con al massimo npt punti: 
            #   fs*nper/ff <= npt con fs=100MHz/df, df intero.
            #   df = ceil(100MHz*nper/(npt*ff))
            df = math.ceil(100e6 * nper / (npt * ff))
            scope.fs = 100e6 / df
            scope.npt = int(scope.fs * nper / ff)

            # Trigger e frequenza
            scope.trig(True, hist=0.01)
            wavegen.w1.freq = ff

            # [3c] Campionamento e analisi risultati
            scope.sample()
            fitfunc = lambda x, A, phi, offset: A * np.cos(2 * np.pi * ff * x + phi) + offset
            pp1, cm1 = so.curve_fit(fitfunc, scope.time.vals, scope.ch1.vals, p0=[1, 0, 0])
            pp2, cm2 = so.curve_fit(fitfunc, scope.time.vals, scope.ch2.vals, p0=[1, 0, 0])

            # [3d] Aggiustamento della fase (se ampiezza negativa...)
            if pp1[0] < 0:
                pp1[0] *= -1
                pp1[1] += np.pi
            if pp2[0] < 0:
                pp2[0] *= -1
                pp2[1] += np.pi

            # [3e] Aggiornamento dei dati (solo andata in colonna 0)
            Am[findex, ar] = pp2[0] / pp1[0]
            # fase in [-pi, pi]
            phim[findex, ar] = (pp2[1] - pp1[1] + np.pi) % (2 * np.pi) - np.pi

            # [3f] Aggiornamento plots
            if flag_first:
                flag_first = False
                if flag_show:
                    hp1, = ax1.plot(1000 * scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp2, = ax2.plot(1000 * scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    ax1.grid(True)
                    ax2.grid(True)
                    ax1.set_xticklabels([])
                    ax1.set_ylabel("Ch1 [V]", fontsize=15)
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Ch2 [V]", fontsize=15)
                    ax1.set_xlim([0, nper / ff])
                    ax2.set_xlim([0, nper / ff])
                    ax1.set_title(f"Starting")
                hp3A, = ax3.loglog(fv, Am[:, 0], ".", markerfacecolor="none", label="Amp go", color="tab:orange")
                hp4A, = ax4.semilogx(fv, phim[:, 0], ".", markerfacecolor="none", label="phi go", color="tab:orange")
                if flag_return:
                    hp3R, = ax3.loglog(fv, Am[:, 1], "v", markerfacecolor="none", label="Amp return", color="tab:blue")
                    hp4R, = ax4.semilogx(fv, phim[:, 1], "v", markerfacecolor="none", label="phi return", color="tab:blue")
                ax3.grid(True)
                ax4.grid(True)
                ax3.set_xticklabels([])
                ax3.yaxis.tick_right()
                ax3.yaxis.set_label_position('right')
                ax3.set_ylabel("Gain [pure]", fontsize=15)
                ax4.set_xlabel("Freq [Hz]", fontsize=15)
                ax4.yaxis.tick_right()
                ax4.yaxis.set_label_position('right')
                ax4.set_yticks([-np.pi, -np.pi/2, 0, np.pi/2, np.pi])
                ax4.set_yticklabels(["-180", "-90", "0", "+90", "+180"])
                ax4.set_ylabel("Phase [deg]", fontsize=15)
                ax3.legend()
                ax4.legend()
                plt.tight_layout()
                plt.show(block=False)
            else:
                # Aggiorna info sul sampling
                if ff > 1e3:
                    title = f"{ff/1e3:.1f}kHz"
                else:
                    title = f"{ff:.1f}Hz"
                if scope.fs > 1e6:
                    ax1.set_title(title + f" [{scope.npt:d}pts @ {scope.fs/1e6:.1f}MSa/s, ]")
                elif scope.fs > 1e3:
                    ax1.set_title(title + f" [{scope.npt:d}pts @ {scope.fs/1e3:.1f}kSa/s, ]")
                else:
                    ax1.set_title(title + f" [{scope.npt:d}pts @ {scope.fs:.1f}Sa/s, ]")

                # Aggiorna i dati...
                if flag_show:
                    hp1.set_xdata(1000 * scope.time.vals)
                    hp1.set_ydata(scope.ch1.vals)
                    hp2.set_xdata(1000 * scope.time.vals)
                    hp2.set_ydata(scope.ch2.vals)
                    ax1.set_xlim([-500 * nper / ff, +500 * nper / ff])
                    ax2.set_xlim([-500 * nper / ff, +500 * nper / ff])
                    mi = scope.ch2.vals.min()
                    ma = scope.ch2.vals.max()
                    dm = ma - mi
                    m0 = (ma + mi) / 2
                    ax2.set_ylim([m0 - 0.6 * dm, m0 + 0.6 * dm])
                if ar == 0:
                    hp3A.set_ydata(Am[:, 0])
                    hp4A.set_ydata(phim[:, 0])
                    ax3.set_xlim([f0, ff])
                    # limiti robusti per ampiezza se presenti dati validi
                    valid_amp = Am[:, 0][~np.isnan(Am[:, 0])]
                    if valid_amp.size > 0:
                        ax3.set_ylim([0.5 * np.min(valid_amp), 2 * np.max(valid_amp)])
                    ax4.set_xlim([f0, ff])
                else:
                    hp3R.set_ydata(Am[:, 1])
                    hp4R.set_ydata(phim[:, 1])
                fig.canvas.draw()
                fig.canvas.flush_events()

    # [4] Salvataggio dati grezzi per questa acquisizione (solo "andata")
    matrice_risultati_ampiezza.append(Am[:, 0].copy())
    matrice_risultati_fase.append(phim[:, 0].copy())

# Converto le liste in matrici
matrice_finale = np.array(matrice_risultati_ampiezza)      # (N_acquisizioni, nf)
matrice_finale_fase = np.array(matrice_risultati_fase)     # (N_acquisizioni, nf)

# Stampa le dimensioni per verifica
print("\n--- Acquisizioni completate ---")
print(f"Dimensioni della matrice ampiezze: {matrice_finale.shape}")
print(f"Dimensioni della matrice fasi:     {matrice_finale_fase.shape}")

print("\n--- Inizio elaborazione statistica ---")
# Ampiezza: media e varianza della media (campionaria/N_acquisizioni)
medie_ampiezze = np.nanmean(matrice_finale, axis=0)
varianze_ampiezze = np.nanvar(matrice_finale, axis=0, ddof=1) / N_acquisizioni

# Fase: media/errore con statistica circolare
medie_fasi, varianze_fasi = circolare_media_var_della_media(matrice_finale_fase)

# ==========================================================================
#  SALVATAGGIO DEI DATI PROCESSATI

# ----- GUADAGNO -----
dati_processati_gain = np.column_stack((fv, medie_ampiezze, varianze_ampiezze))
header_gain = (
    "Dati processati dal Bode Plotter (GUADAGNO)\n"
    "Colonna 1: Frequenza [Hz]\n"
    f"Colonna 2: Ampiezza Media (su {N_acquisizioni} acquisizioni)\n"
    f"Colonna 3: Varianza campionaria della media (su {N_acquisizioni} acquisizioni)"
)
np.savetxt("Dati_bode_gain_mcp601_1000.txt", dati_processati_gain, fmt='%.8e', header=header_gain)
print("Dati guadagno salvati in 'Dati_bode_gain_mcp601.txt'")

# ----- FASE (radianti) -----
dati_processati_phase = np.column_stack((fv, medie_fasi, varianze_fasi))
header_phase = (
    "Dati processati dal Bode Plotter (FASE, radianti)\n"
    "Colonna 1: Frequenza [Hz]\n"
    f"Colonna 2: Fase media circolare (su {N_acquisizioni} acquisizioni) [rad]\n"
    "Colonna 3: Varianza campionaria della media (circolare) [rad^2]"
)
np.savetxt("Dati_bode_phase_mcp601_1000.txt", dati_processati_phase, fmt='%.8e', header=header_phase)
print("Dati fase salvati in 'Dati_bode_phase_mcp601.txt'")

# ---------------------------------------
ad2.close()
print("Script terminato.")


Dispositivo #1 [SN:210321B5D136, hdwf=1] connesso!
Configurazione #1
--- Acquisizione 1 di 10 ---
--- Acquisizione 2 di 10 ---


C:\Users\aless\AppData\Local\Temp\ipykernel_32280\587301548.py:205: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax3.set_xlim([f0, ff])
C:\Users\aless\AppData\Local\Temp\ipykernel_32280\587301548.py:210: UserWarning: Attempting to set identical low and high xlims makes transformation singular; automatically expanding.
  ax4.set_xlim([f0, ff])


--- Acquisizione 3 di 10 ---
--- Acquisizione 4 di 10 ---
--- Acquisizione 5 di 10 ---
--- Acquisizione 6 di 10 ---
--- Acquisizione 7 di 10 ---
--- Acquisizione 8 di 10 ---
--- Acquisizione 9 di 10 ---
--- Acquisizione 10 di 10 ---

--- Acquisizioni completate ---
Dimensioni della matrice ampiezze: (10, 200)
Dimensioni della matrice fasi:     (10, 200)

--- Inizio elaborazione statistica ---
Dati guadagno salvati in 'Dati_bode_gain_mcp601.txt'
Dati fase salvati in 'Dati_bode_phase_mcp601.txt'
Dispositivo disconnesso.
Script terminato.


In [ ]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as so
import math
# === PARALLEL ===
import multiprocessing as mp


# === PARALLEL ===
# Worker di fit (solo CPU; nessun accesso all'hardware o a matplotlib)
def _fit_one(args):
    findex, ar, ff, tvals, ch1vals, ch2vals = args

    def fitfunc(x, A, phi, offset):
        return A * np.cos(2*np.pi*ff*x + phi) + offset

    pp1, _ = so.curve_fit(fitfunc, tvals, ch1vals, p0=[1, 0, 0])
    pp2, _ = so.curve_fit(fitfunc, tvals, ch2vals, p0=[1, 0, 0])

    # Aggiustamento fase se ampiezza negativa
    if pp1[0] < 0:
        pp1[0] *= -1
        pp1[1] += np.pi
    if pp2[0] < 0:
        pp2[0] *= -1
        pp2[1] += np.pi

    A_ratio = pp2[0] / pp1[0]
    phi = (pp2[1] - pp1[1] + np.pi) % (2*np.pi) - np.pi
    return (findex, ar, A_ratio, phi)


if __name__ == "__main__":
    # ==========================================================================
    #  Parametri dello script
    nper = 2            # numero periodi usati per la stima   
    npt = 8192          # numero MASSIMO di punti acquisiti
    nf = 200            # numero di frequenze nello sweep da f0 a f1   
    f0 = 50        
    f1 = 500e3      
    flag_return = False  # A/R? o solo andata?
    flag_show = True    # Visualizza i segnali nel tempo? (è uguale...)
    # vettore delle frequenze
    fv = np.logspace(np.log10(f0), np.log10(f1), nf) 

    # ==========================================================================
    #  Configurazione base AD2 (più parametri impostati dopo)
    ad2 = tdwf.AD2()
    ad2.vdd = 3
    ad2.vss = -3
    ad2.power(True)

    wavegen = tdwf.WaveGen(ad2.hdwf)
    wavegen.w1.ampl = 0.02
    wavegen.w1.func = tdwf.funcSine 
    wavegen.w1.start()
    scope = tdwf.Scope(ad2.hdwf)
    scope.ch1.rng = 5
    scope.ch2.rng = 50
    scope.ch1.avg = True
    scope.ch2.avg = True

    # ==========================================================================
    #   Esecuzione...

    # [1] Crea plot
    fig, [[ax1, ax3], [ax2, ax4]] = plt.subplots(2,2,figsize=(12,6),                                
        gridspec_kw={'width_ratios': [1, 2]})
    fig.canvas.manager.set_window_title('Spazzata frequenza')
    # [2] Preparazione dataset
    flag_first = True                # primo plot diverso dai seguenti (update)
    Am = np.full((nf, 2), np.nan)    # pre-allocazione dati (nan non sono plottati!) 
    phim = np.full((nf, 2), np.nan)  # pre-allocazione dati

    # === PARALLEL ===
    # Contenitore per i dati grezzi delle misure (da elaborare in parallelo)
    # Ogni elemento: (findex, ar, ff, tvals, ch1vals, ch2vals)
    tasks = []

    # [3] Ciclo misura: SOLO acquisizione (nessuna stima/fit qui)
    for ar in range(2 if flag_return else 1): # Ciclo A/R
        for ii in range(len(fv)):  # Ciclo frequenze
            # [3a] calcolo frequenza
            if ar==0: # caso A(ndata)
                findex = ii
            else:     # caso R(itorno)
                findex = len(fv)-ii-1
            # Frequenza attuale
            ff = fv[findex]
            # [3b] stima parametri di sampling
            df = math.ceil(100e6*nper/(npt*ff))
            scope.fs = 100e6/df
            scope.npt = int(scope.fs*nper/ff)
            #  Ribadiamo il trigger... 
            scope.trig(True, hist = 0.01)
            wavegen.w1.freq = ff 
            # [3c] Campionamento
            scope.sample()

            # === PARALLEL ===
            # Copia dei dati grezzi per il fit parallelo
            tvals  = scope.time.vals.copy()
            ch1val = scope.ch1.vals.copy()
            ch2val = scope.ch2.vals.copy()
            tasks.append((findex, ar, ff, tvals, ch1val, ch2val))

            # [3f] Aggiornamento plots (SOLO tempo, per non dipendere dai fit)
            if flag_first:
                flag_first = False
                if flag_show:
                    hp1, = ax1.plot(1000*scope.time.vals, scope.ch1.vals, "-", label="Ch1", color="tab:orange")
                    hp2, = ax2.plot(1000*scope.time.vals, scope.ch2.vals, "-", label="Ch2", color="tab:blue")
                    ax1.grid(True)
                    ax2.grid(True)
                    ax1.set_xticklabels([])
                    ax1.set_ylabel("Ch1 [V]", fontsize=15)
                    ax2.set_xlabel("Time [msec]", fontsize=15)
                    ax2.set_ylabel("Ch2 [V]", fontsize=15)
                    ax1.set_xlim([0, nper/ff])
                    ax2.set_xlim([0, nper/ff])
                    ax1.set_title(f"Starting")
                # Prepara i placeholder dei grafici Bode (si aggiorneranno DOPO i fit)
                hp3A, = ax3.loglog(fv, Am[:, 0], ".", markerfacecolor = "none", label="Amp go", color="tab:orange")
                hp4A, = ax4.semilogx(fv, phim[:, 0], ".",  markerfacecolor = "none", label="phi go", color="tab:orange")
                if flag_return:
                    hp3R, = ax3.loglog(fv, Am[:, 1], "v",  markerfacecolor = "none", label="Amp return", color="tab:blue")
                    hp4R, = ax4.semilogx(fv, phim[:, 1], "v",  markerfacecolor = "none", label="phi return", color="tab:blue")
                ax3.grid(True)
                ax4.grid(True)            
                ax3.set_xticklabels([])
                ax3.yaxis.tick_right()
                ax3.yaxis.set_label_position('right')
                ax3.set_ylabel("Gain [pure]", fontsize=15)
                ax4.set_xlabel("Freq [Hz]", fontsize=15)
                ax4.yaxis.tick_right()
                ax4.yaxis.set_label_position('right')
                ax4.set_yticks([-np.pi,-np.pi/2,0,np.pi/2,np.pi])
                ax4.set_yticklabels(["-180","-90","0","+90","+180"])
                ax4.set_ylabel("Phase [deg]", fontsize=15)
                ax3.legend()
                ax4.legend()
                plt.tight_layout()
                plt.show(block=False)    
            else:
                # Aggiorna titolo/limiti del tempo (facoltativo, non impatta la parallelizzazione)
                if ff > 1e3:
                    title = f"{ff/1e3:.1f}kHz"
                else:
                    title = f"{ff:.1f}Hz"
                if scope.fs > 1e6:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e6:.1f}MSa/s, ]")
                elif scope.fs > 1e3:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs/1e3:.1f}kSa/s, ]")
                else:
                    ax1.set_title(title+f" [{scope.npt:d}pts @ {scope.fs:.1f}Sa/s, ]")
                if flag_show:
                    hp1.set_xdata(1000*scope.time.vals)
                    hp1.set_ydata(scope.ch1.vals)
                    hp2.set_xdata(1000*scope.time.vals)
                    hp2.set_ydata(scope.ch2.vals)
                    ax1.set_xlim([-500*nper/ff, +500*nper/ff])
                    ax2.set_xlim([-500*nper/ff, +500*nper/ff])
                    mi = scope.ch2.vals.min()
                    ma = scope.ch2.vals.max()
                    dm = ma-mi
                    m0 = (ma+mi)/2
                    ax2.set_ylim([m0-0.6*dm,m0+0.6*dm])
                fig.canvas.draw()
                fig.canvas.flush_events()

    # === PARALLEL ===
    # FASE 2: elaborazione in parallelo (fit)
    # Nota: nessun accesso all'hardware o a matplotlib nei worker
    with mp.Pool(processes=mp.cpu_count()) as pool:
        # imap_unordered per sfruttare meglio i core; ricomponiamo poi per indice
        for findex, ar, A_ratio, phi in pool.imap_unordered(_fit_one, tasks):
            Am[findex, ar] = A_ratio
            phim[findex, ar] = phi

    # === PARALLEL ===
    # Aggiorna i grafici Bode una volta conclusi i fit
    if 'hp3A' in locals():
        hp3A.set_ydata(Am[:, 0])
        hp4A.set_ydata(phim[:, 0])
        if 'hp3R' in locals() and flag_return:
            hp3R.set_ydata(Am[:, 1])
            hp4R.set_ydata(phim[:, 1])

        ax3.set_xlim([f0, f1])
        ax4.set_xlim([f0, f1])

        # Limiti Y robusti (ignorando NaN)
        if np.any(np.isfinite(Am[:, 0])):
            y_min = 0.5 * np.nanmin(Am[:, 0])
            y_max = 2.0 * np.nanmax(Am[:, 0])
            if np.isfinite(y_min) and np.isfinite(y_max) and (y_max > y_min) and (y_min > 0):
                ax3.set_ylim([y_min, y_max])

        fig.canvas.draw()
        fig.canvas.flush_events()

    # Salvataggio risultati come prima
    data = np.column_stack((fv, Am, phim))
    np.savetxt("AD8031_Gain1e3.txt", data)

    # ---------------------------------------
    ad2.close()
